# Semantic Segmentation
1. by : Anubhav Singh
2. [Kaggle Link](https://www.kaggle.com/anubhav1302/semantic-segmentation)

In [3]:
import numpy as np
import os
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
from skimage import io
from glob import glob
from tqdm import tqdm

## Set-up

1. Get the paths of both the training set & validation set
2. Load the images to use w/ func "def load_imgs()"


### Questions 
1. Why do I have to run this everytime I open the notebook?
2. What's masks?
3. Why normalize?

In [4]:
train_path = 'data/train'
val_path = 'data/val'

In [5]:
train_imgs = []
train_masks = []

val_imgs = []
val_masks = []

def load_imgs(path) :
    temp_img = []
    temp_mask = []
    
    imgs = glob(os.path.join(path, '*jpg'))
    
    for image in tqdm(imgs) :
        image = cv2.imread(image)
        
#         cv.normalize(src, dst[, alpha[, beta[, norm_type[, dtype[, mask]]]]]) -> dst
        image = cv2.normalize(image, None, 0, 1, cv2.NORM_MINMAX, cv2.CV_32F)
        img = image[:, : 256]
        msk = image[:, 256 :]
        
        temp_img.append(img)
        temp_mask.append(msk)
        
    return temp_img, temp_mask

train_imgs, train_masks = load_imgs(train_path)
val_imgs, val_masks = load_imgs(val_path)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 561.01it/s]


# Placeholders
1. tf.placeholder 
    1. dtype, tf.
    2. shape, [_, w, h, d]
    3. name, name = ''

In [6]:
image = tf.compat.v1.placeholder(tf.float32, [None, 256, 256, 3], name = 'Input_image')
mask = tf.compat.v1.placeholder(tf.float32, [None, 256, 256, 3], name = 'Input_mask')

# Hidden Layer Functions

1. relu
2. convolution
3. max pooling
3. convolution transpose 

In [7]:
def lrelu(x, threshold = 0.1) :
    get_max = tf.maximum(x, x * threshold)
    return get_max

def conv_layer(x, n_filts, kern_size, strd, padding='SAME') :
    x = tf.layers.conv2d(x, filters = n_filts, kernel_size = kern_size, strides = strd, padding = padding)
    x = tf.nn.relu(x)
    return x

# diffs from his
def max_pooling(x, pool_size) :
    x = tf.layers.max_pooling2d(x, pool_size = pool_size)
    return x

def conv_trans(x, n_filts, kern_size, strd, padding='SAME') :
    x = tf.layers.conv2d_transpose(x, filters = n_filts, kernel_size = kern_size, strides = strd, padding = padding)
    x = tf.nn.relu(x)
    return x

 # Beta Network
 1. conda install gast==0.2.2 for below to work

In [8]:
k_s = 4
conv_strd = 1

p_s = 2
pool_strd = 2

In [9]:
# Branch 0
layer_1_b0 = conv_layer(image, n_filts = 64, kern_size = k_s, strd = conv_strd)
mp_1_b0 = tf.layers.max_pooling2d(layer_1_b0, pool_size = p_s, strides = pool_strd)

# print("Layer 1 shapes : \n>>layer 1 : ", layer_1_b0.shape, "\n>>pooling layer : ", mp_1_b0.shape)
    
layer_2_b0 = conv_layer(mp_1_b0, n_filts = 128, kern_size = k_s, strd = conv_strd)
mp_2_b0 = tf.layers.max_pooling2d(layer_2_b0, pool_size = p_s, strides = pool_strd)

# print("\nLayer 1 shapes : \n>>layer 2 : ", layer_2_b0.shape, "\n>>pooling layer : ", mp_2_b0.shape)

layer_3_b0 = conv_layer(mp_2_b0, n_filts = 256, kern_size = k_s, strd = conv_strd)
mp_3_b0 = tf.layers.max_pooling2d(layer_3_b0, pool_size = p_s, strides = pool_strd)

# print("\nLayer 1 shapes : \n>>layer 3 : ", layer_3_b0.shape, "\n>>pooling layer : ", mp_3_b0.shape)

layer_4_b0 = conv_layer(mp_3_b0, n_filts = 512, kern_size = k_s, strd = conv_strd)
mp_4_b0 = tf.layers.max_pooling2d(layer_4_b0, pool_size = p_s, strides = pool_strd)

# print("\nLayer 1 shapes : \n>>layer 4 : ", layer_4_b0.shape, "\n>>pooling layer : ", mp_4_b0.shape)

layer_5_b0 = conv_layer(mp_4_b0, n_filts = 1024, kern_size = k_s, strd = conv_strd)
mp_5_b0 = tf.layers.max_pooling2d(layer_5_b0, pool_size = p_s, strides = pool_strd)

# print("\nLayer 1 shapes : \n>>layer 5 : ", layer_5_b0.shape, "\n>>pooling layer : ", mp_5_b0.shape)


Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use keras.layers.MaxPooling2D instead.


In [10]:
# Branch 1
layer_b1 = conv_layer(image, n_filts = 128, kern_size = k_s, strd = conv_strd)
mp_b1 = tf.layers.max_pooling2d(layer_b1, pool_size = p_s, strides = pool_strd)

beta_1 = tf.keras.layers.add([layer_2_b0, mp_b1])

layer_b2 = conv_layer(beta_1, n_filts = 256, kern_size = k_s, strd = conv_strd)
mp_b2 = tf.layers.max_pooling2d(layer_b2, pool_size = p_s, strides = pool_strd)

beta_2 = tf.keras.layers.add([layer_3_b0, mp_b2])

layer_b3 = conv_layer(beta_2, n_filts = 512, kern_size = k_s, strd = conv_strd)
mp_b3 = tf.layers.max_pooling2d(layer_b3, pool_size = p_s, strides = pool_strd)

beta_3 = tf.keras.layers.add([layer_4_b0, mp_b3])

layer_b4 = conv_layer(beta_3, n_filts = 1024, kern_size = k_s, strd = conv_strd)
mp_b4 = tf.layers.max_pooling2d(layer_b4, pool_size = p_s, strides = pool_strd)

beta_4 = tf.keras.layers.add([layer_5_b0, mp_b4])
beta_0 = layer_1_b0

In [11]:
# Downsample : 64, 128, 256, 512, 1024
x_layer_1 = conv_layer(image, n_filts = 64, kern_size = 5, strd = 1)
x_layer_1 = conv_layer(x_layer_1, n_filts = 64, kern_size = 4, strd = 1)
x_layer_1 = conv_layer(x_layer_1, n_filts = 64, kern_size = 4, strd = 2)
x_batch_1 = tf.layers.batch_normalization(x_layer_1)

# print("Size of 64 x_batch_1 : \n", x_batch_1.shape)

x_layer_2 = conv_layer(x_batch_1, n_filts = 128, kern_size = 5, strd = 1)
x_layer_2 = conv_layer(x_layer_2, n_filts = 128, kern_size = 4, strd = 1)
x_layer_2 = conv_layer(x_layer_2, n_filts = 128, kern_size = 4, strd = 2)
x_batch_2 = tf.layers.batch_normalization(x_layer_2)

# print("\nSize of 128 x_batch_2 : \n", x_batch_2.shape)

x_layer_3 = conv_layer(x_batch_2, n_filts = 256, kern_size = 5, strd = 1)
x_layer_3 = conv_layer(x_layer_3, n_filts = 256, kern_size = 4, strd = 1)
x_layer_3 = conv_layer(x_layer_3, n_filts = 256, kern_size = 4, strd = 2)
x_batch_3 = tf.layers.batch_normalization(x_layer_3)

# print("\nSize of 256 x_batch_3 : \n", x_batch_3.shape)

x_layer_4 = conv_layer(x_batch_3, n_filts = 512, kern_size = 5, strd = 1)
x_layer_4 = conv_layer(x_layer_4, n_filts = 512, kern_size = 4, strd = 1)
x_layer_4 = conv_layer(x_layer_4, n_filts = 512, kern_size = 4, strd = 2)
x_batch_4 = tf.layers.batch_normalization(x_layer_4)

# print("\nSize of 512 x_batch_4 : \n", x_batch_4.shape)

x_layer_5 = conv_layer(x_batch_4, n_filts = 1024, kern_size = 4, strd = 1)
x_layer_5 = conv_layer(x_batch_4, n_filts = 1024, kern_size = 4, strd = 8)
x_batch_5 = tf.layers.batch_normalization(x_layer_5)

# print("\nSize of 1024 x_batch_5 : \n", x_batch_5.shape)

Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).


In [12]:
# Upsample : 1024, 512, 256, 128, 64
y_layer_1 = conv_trans(x_batch_5, n_filts = 1024, kern_size = 4, strd = 8)
y_layer_1 = tf.keras.layers.add([y_layer_1, beta_4])
y_layer_1 = conv_layer(y_layer_1, n_filts = 1024, kern_size = 4, strd = 1)
y_batch_1 = tf.layers.batch_normalization(y_layer_1)

y_layer_2 = conv_trans(y_batch_1, n_filts = 512, kern_size = 5, strd = 2)
y_layer_2 = tf.keras.layers.add([y_layer_2, beta_3])
y_layer_2 = conv_layer(y_layer_2, n_filts = 512, kern_size = 4, strd = 1)
y_layer_2 = conv_layer(y_layer_2, n_filts = 512, kern_size = 4, strd = 1)
y_batch_2 = tf.layers.batch_normalization(y_layer_2)

y_layer_3 = conv_trans(y_batch_2, n_filts = 256, kern_size = 2, strd = 2)
y_layer_3 = tf.keras.layers.add([y_layer_3, beta_2])
y_layer_3 = conv_layer(y_layer_3, n_filts = 256, kern_size = 4, strd = 1)
y_layer_3 = conv_layer(y_layer_3, n_filts = 256, kern_size = 4, strd = 1)
y_batch_3 = tf.layers.batch_normalization(y_layer_3)

y_layer_4 = conv_trans(y_batch_3, n_filts = 128, kern_size = 3, strd = 2)
y_layer_4 = tf.keras.layers.add([y_layer_4, beta_1])
y_layer_4 = conv_layer(y_layer_4, n_filts = 128, kern_size = 4, strd = 1)
y_layer_4 = conv_layer(y_layer_4, n_filts = 128, kern_size = 4, strd = 1)
y_batch_4 = tf.layers.batch_normalization(y_layer_4)

y_layer_5 = conv_trans(y_batch_4, n_filts = 64, kern_size = 2, strd = 2)
y_layer_5 = tf.keras.layers.add([y_layer_5, beta_0])
y_layer_5 = conv_layer(y_layer_5, n_filts = 64, kern_size = 4, strd = 1)
y_layer_5 = conv_layer(y_layer_5, n_filts = 64, kern_size = 4, strd = 1)
y_batch_5 = tf.layers.batch_normalization(y_layer_5)

Instructions for updating:
Use `tf.keras.layers.Conv2DTranspose` instead.


In [13]:
out = tf.layers.conv2d(y_batch_5, activation = None, filters = 3, kernel_size = 1, strides = 1, padding = 'SAME')

In [14]:
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=mask,logits=out))
train_op = tf.compat.v1.train.AdamOptimizer(learning_rate=0.0001).minimize(loss)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [15]:
num_epochs = 100
batch_size = 25
train_batches = len(train_imgs) // batch_size
val_batches = len(val_imgs) // batch_size
train_loss,val_loss = [],[]
saver = tf.compat.v1.train.Saver()

In [ ]:
with tf.compat.v1.Session() as sess:
    
    sess.run(tf.compat.v1.global_variables_initializer())
    
    for epoch in range(num_epochs):
        print('======================================')
        print('Epoch: ',(epoch+1))
        print('In Training..')
        
        for batch in tqdm(range(train_batches)):
            train_img_batch = train_imgs[batch * batch_size:(batch + 1) * batch_size]
            train_msk_batch = train_masks[batch * batch_size:(batch + 1) * batch_size]
            
            #reshape images and masks
            train_img_batch = np.reshape(train_img_batch,(len(train_img_batch), 256, 256, 3))
            train_msk_batch = np.reshape(train_msk_batch,(len(train_msk_batch), 256, 256, 3))
            t_loss,_ = sess.run([loss,train_op],feed_dict = {
                    image:train_img_batch,mask:train_msk_batch})
            
        print('In Validation..')
        for v_batch in tqdm(range(val_batches)):
            val_img_batch = val_images[v_batch * batch_size:(v_batch + 1) *batch_size]
            val_msk_batch = val_masks[v_batch * batch_size:(v_batch + 1) * batch_size]
            
            #Reshape batches
            val_img_batch = np.reshape(val_img_batch, (len(val_img_batch), 256, 256, 3))
            val_msk_batch = np.reshape(val_msk_batch, (len(val_msk_batch), 256, 256, 3))
            v_loss,_ = sess.run([loss,train_op],feed_dict = {image:val_img_batch, mask:val_msk_batch})
        train_loss.append(t_loss)
        val_loss.append(v_loss)
        
        print('Train Loss: ', t_loss)
        print('Val Loss: ', v_loss)
    saver.save(sess, "model.ckpt")

2022-02-18 11:06:54.634350: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-18 11:06:54.634974: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 16. Tune using inter_op_parallelism_threads for best performance.
  0%|                                                                                                                    | 0/119 [00:00<?, ?it/s]

Epoch:  1
In Training..
